# CROP FACES FROM IMAGE DATASET

##  Notebook Description

First of all, in order to classify the images we are going to crop all the faces that appears in our images.


After that, we will resize these images.

## Libraries

In [ ]:
# ///Basic libraries
import numpy as np
import pandas as pd
import random

# ///Path files and xml manipulation
from os.path import isfile, join
import xmltodict
import os
from os import listdir

# // Image processing
import cv2

# ///Visualization
import matplotlib.pyplot as plt
import seaborn as sns

#/////////////////TENSORFLOW LIBRARIES OK
#import tensorflow as tf
#from tensorflow.keras.utils import Progbar
#from tensorflow.keras.models import Model, Sequential
#from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, GlobalMaxPooling2D, Input
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras import applications


#import glob
#import math


%matplotlib inline

## Functions

In [ ]:
def getImageNames():
    
    """
    This function give us all the image names of our dataset
    """
    
    image_names = []
    
    for dirname, _, filenames in os.walk('../ih-final-project/data/'):
        for filename in filenames:
            fullpath = os.path.join(dirname, filename)
            extension = fullpath[len(fullpath) - 4:]
            if extension != '.xml':
                image_names.append(filename)
                
    return image_names


In [ ]:
image_names

In [ ]:
def get_path(image_name):
    """
    This function extract the image and label path to reassign the same name to both files.
    And return the new image and label path of the images.
    
    """
    
    original_path = '../data/medical-masks-dataset/images/'
    image_path = original_path + 'images/' + image_name
    
    #we have .jpeg .jpg .png
    if image_name[-4:] == 'jpeg':
        label_name = image_name[:-5] + '.xml'
    else:
        label_name = image_name[:-4] + '.xml'
    
    label_path = original_path + 'labels/' + label_name
        
    return  image_path, label_path

In [ ]:
def parse_xml(label_path):
    
    """
    With this function we can obtain the necessary information of each image.
    XML files contain their descriptions like their locations and labels good, none or bad.
    
    """
    
    x = xmltodict.parse(open(label_path , 'rb'))
    item_list = x['annotation']['object']
    
  
    if not isinstance(item_list, list):
        item_list = [item_list]
    #we use isinstance if we only have one person in the image.
    
    result = []
    
    for item in item_list:
        name = item['name']
        findfacebox = [(int(item['findfacebox']['xmin']), int(item['findfacebox']['ymin'])),
                  (int(item['findfacebox']['xmax']), int(item['findfacebox']['ymax']))] 
        
        result.append((name, findfacebox))
    
    size = [int(x['annotation']['size']['width']), 
            int(x['annotation']['size']['height'])]
    
    return result, size


In [ ]:
def visualize_image(image_name, findfacebox=True):
    
    """
    This function help us to visualize the images that we want with a common size.
    """
    
    image_path, label_path = get_path(image_name)
    
    image = cv2.imread(image_path)
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    if findfacebox:
        labels, size = parse_xml(label_path)
        thickness = int(sum(size)/400.)
        
        for label in labels:
            name, findfacebox = label
            
            if name == 'good':
                cv2.rectangle(image, 
                              findfacebox[0], 
                              findfacebox[1], 
                              (0, 255, 0), thickness)
            elif name == 'bad':
                cv2.rectangle(image, 
                              findfacebox[0], 
                              findfacebox[1], 
                              (255, 0, 0), 
                              thickness)
            # we only have two cases, good or bad, but equally we take care if exists some incorrect image
            else:
                cv2.rectangle(image, findfacebox[0], findfacebox[1], (0, 0, 255), thickness)
    
    plt.figure(figsize=(20, 20))
    plt.subplot(1, 2, 1)
    plt.axis('off')
    plt.title(image_name)
    plt.imshow(image)
    plt.show()

In [ ]:
image_names = getImageNames()

In [ ]:
n = 5

for i in range(n):
     visualize_image(image_names[i])

In [ ]:
#This error happened because the image didn't load properly